In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch

REBUILD_DATA = True

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

class animals():
    IMG_SIZE = 50
    Cats = "data/Cat"
    Dogs = "data/Dog"
    Foxs = "data/Fox"
    training_Data = []
    Labels = {Cats : 0 , Dogs : 1,Foxs:2}
    counter = {Cats : 0 , Dogs : 0,Foxs:0}    
    
    def importData(self):
        for label in self.Labels:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label,f)
                    img = cv2.imread(path,
                                     cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img,
                                     (self.IMG_SIZE,self.IMG_SIZE))
                    self.training_Data.append([np.array(img), 
                                               np.eye(3)[self.Labels[label]]])
                    if label == self.Cats:
                        counter["Cats"]+=1
                    elif label == self.Dogs:
                        counter["Dogs"]+=1
                    elif label == self.Foxs:
                        counter["Foxs"]+=1
                    
                except Exception as e :
                    pass
        
        np.random.shuffle(self.training_Data)
        np.save("training_Data.npy",self.training_Data)
                
            
if REBUILD_DATA:
    animal = animals()
    animal.importData()
    

  0%|▏                                                                             | 39/12502 [00:00<00:32, 384.22it/s]

Running on the GPU


100%|███████████████████████████████████████████████████████████████████████████████| 665/665 [00:05<00:00, 118.47it/s]
C:\Users\USER\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [4]:
training_Data = np.load("training_Data.npy",allow_pickle=True)

In [5]:
len(training_Data)

25611

In [6]:
training_Data[0][1]

array([0., 1., 0.])

In [7]:
 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Conv2d(input,features,size)
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(51200,512)
        self.fc2 = nn.Linear(512,3)
    
    def convs(self,x):
        
        x =  F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x =  F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x =  F.max_pool2d(F.relu(self.conv3(x)),(2,2))
    
        return F.softmax(x,dim=1) 
    
    def forward(self,x):
        try : 
            x = self.convs(x)
            # flatten

            x = torch.flatten(x)
            
            self.fc1 = nn.Linear(x.size()[0],512)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x
        except Exception as e:
            print(e)
            return e
        
net = Net()     

In [8]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()

x = torch.Tensor([i[0] for i in training_Data]).view(-1,50,50)
x = x/255.0
y = torch.Tensor([i[1] for i in training_Data])
    
VAL_PCT = 0.1
val_size = int(len(x)*VAL_PCT)


In [9]:
train_x = x[:-val_size]
train_y = y[:-val_size]

test_x = x[-val_size:]
test_y = y[-val_size:]

In [10]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0,len(train_x),BATCH_SIZE)):
        try:
            batch_x = train_x[i:i+BATCH_SIZE].view(-1,1,50,50)
            batch_y = train_y[i:i+BATCH_SIZE]
            net.zero_grad()
            outputs = net.forward(batch_x)
            print(outputs)
            loss = loss_function(outputs,batch_y)
            loss.backward()
            optimizer.step()
        except Exception as e:
            pass
print(loss)

  0%|                                                                                          | 0/231 [00:00<?, ?it/s]

tensor([-0.0377,  0.0361,  0.0236], grad_fn=<AddBackward0>)


C:\Users\USER\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([100, 3])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|▎                                                                                 | 1/231 [00:00<02:27,  1.56it/s]

tensor([-0.0340,  0.0361,  0.0223], grad_fn=<AddBackward0>)


  1%|▋                                                                                 | 2/231 [00:01<02:22,  1.61it/s]

...

tensor([0.4344, 0.4690, 0.0270], grad_fn=<AddBackward0>)


 42%|██████████████████████████████████▎                                              | 98/231 [01:00<01:31,  1.46it/s]

tensor([0.4558, 0.4781, 0.0061], grad_fn=<AddBackward0>)


 43%|██████████████████████████████████▋                                              | 99/231 [01:00<01:21,  1.63it/s]


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_x))):
        try:
            real_class = torch.argmax(test_y[i])
            net_out = net.forward(test_x[i].view(-1,1,50,50))
            
            predicted_class = torch.argmax(net_out)
 
            print(real_class,net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
            
        except Exception as e:
            print(str(e))
            break

print("Accuracy:",round(correct/total,3))


In [26]:
arr = torch.rand(4,5)

In [28]:
arr.size()[0]

4

In [30]:
arr[-2:]

tensor([[0.8750, 0.9578, 0.3404, 0.3153, 0.3000],
        [0.2464, 0.3774, 0.1818, 0.7264, 0.1102]])